## Giriş

Bu derste şunlar ele alınacaktır:
- Fonksiyon çağrısı nedir ve hangi durumlarda kullanılır
- Azure OpenAI kullanarak nasıl fonksiyon çağrısı oluşturulur
- Bir uygulamaya fonksiyon çağrısının nasıl entegre edileceği

## Öğrenme Hedefleri

Bu dersi tamamladıktan sonra şunları bilecek ve anlayacaksınız:

- Fonksiyon çağrısı kullanmanın amacı
- Azure Open AI Servisi ile Fonksiyon Çağrısı Kurulumu
- Uygulamanızın kullanım senaryosu için etkili fonksiyon çağrıları tasarlama


## Fonksiyon Çağrılarını Anlamak

Bu derste, eğitim girişimimiz için kullanıcıların teknik kursları bulmak amacıyla bir sohbet botu kullanmasını sağlayan bir özellik geliştirmek istiyoruz. Kullanıcıların beceri seviyelerine, mevcut rollerine ve ilgi duydukları teknolojiye uygun kurslar önereceğiz.

Bunu tamamlamak için şu araçların bir kombinasyonunu kullanacağız:
 - Kullanıcıya sohbet deneyimi sunmak için `Azure Open AI`
 - Kullanıcının isteğine göre kurs bulmasına yardımcı olmak için `Microsoft Learn Catalog API`
 - Kullanıcının sorgusunu alıp API isteği yapmak için bir fonksiyona göndermek amacıyla `Function Calling`

Başlamak için, ilk etapta neden fonksiyon çağrısı kullanmak isteyebileceğimize bakalım:

print("Sonraki istekteki mesajlar:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # GPT'den, fonksiyonun yanıtını görebileceği yeni bir yanıt al


print(second_response.choices[0].message)


### Neden Function Calling

Bu kurstaki başka bir dersi tamamladıysanız, Büyük Dil Modelleri'nin (LLM'ler) gücünü muhtemelen anlamışsınızdır. Umarım aynı zamanda bazı sınırlamalarını da görmüşsünüzdür.

Function Calling, Azure Open AI Service'in şu sınırlamaları aşmak için sunduğu bir özelliktir:
1) Tutarlı yanıt formatı
2) Bir sohbet bağlamında bir uygulamanın diğer kaynaklarından veri kullanabilme yeteneği

Function calling öncesinde, bir LLM'den gelen yanıtlar yapılandırılmamış ve tutarsız oluyordu. Geliştiriciler, her yanıt varyasyonunu işleyebilmek için karmaşık doğrulama kodları yazmak zorundaydı.

Kullanıcılar "Stockholm'de şu an hava nasıl?" gibi sorulara yanıt alamıyordu. Bunun nedeni, modellerin yalnızca eğitildikleri zamana kadar olan verilerle sınırlı olmasıydı.

Aşağıda bu sorunu gösteren bir örneğe bakalım:

Diyelim ki öğrenci verilerinden oluşan bir veritabanı oluşturmak ve onlara en uygun kursu önermek istiyoruz. Aşağıda, içerdiği veriler açısından birbirine çok benzeyen iki öğrenci tanımı var.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Bu veriyi ayrıştırması için bunu bir LLM'ye göndermek istiyoruz. Bu daha sonra uygulamamızda bir API'ye göndermek veya bir veritabanında saklamak için kullanılabilir.

İlgilendiğimiz bilgileri LLM'ye anlatan iki özdeş istem oluşturalım:


Bunu ürünümüz için önemli olan bölümleri ayrıştırması için bir LLM'ye göndermek istiyoruz. Böylece LLM'ye talimat vermek için iki özdeş istem oluşturabiliriz:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Bu iki istemi oluşturduktan sonra, onları `openai.ChatCompletion` kullanarak LLM'ye göndereceğiz. İstemi `messages` değişkeninde saklıyoruz ve role olarak `user` atıyoruz. Bu, bir kullanıcının bir sohbet botuna yazdığı bir mesajı taklit etmek içindir.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

Şimdi her iki isteği de LLM'ye gönderebilir ve aldığımız yanıtı inceleyebiliriz.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Aynı istemler kullanılsa ve açıklamalar benzer olsa da, `Grades` özelliğinin farklı formatlarını alabiliriz.

Yukarıdaki hücreyi birkaç kez çalıştırırsanız, format `3.7` veya `3.7 GPA` olabilir.

Bunun nedeni, LLM'in yazılı istem şeklinde yapılandırılmamış veriler alması ve yine yapılandırılmamış veriler döndürmesidir. Veriyi saklarken veya kullanırken neyle karşılaşacağımızı bilmek için yapılandırılmış bir formata ihtiyacımız var.

Fonksiyonel çağrı kullanarak, bize yapılandırılmış veri döneceğinden emin olabiliriz. Fonksiyon çağrısı kullanıldığında, LLM aslında herhangi bir fonksiyonu çağırmaz veya çalıştırmaz. Bunun yerine, LLM'in yanıtları için takip etmesi gereken bir yapı oluştururuz. Daha sonra bu yapılandırılmış yanıtları, uygulamalarımızda hangi fonksiyonun çalıştırılacağını bilmek için kullanırız.


![Fonksiyon Çağırma Akış Diyagramı](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.tr.png)


### Fonksiyon çağrılarının kullanım alanları

**Harici Araçları Çağırmak**  
Sohbet botları, kullanıcılardan gelen sorulara yanıt vermede oldukça iyidir. Fonksiyon çağrısı kullanılarak, sohbet botları kullanıcı mesajlarını belirli görevleri tamamlamak için kullanabilir. Örneğin, bir öğrenci sohbet botuna "Eğitmenime bu konuda daha fazla yardıma ihtiyacım olduğunu belirten bir e-posta gönder" diyebilir. Bu durumda `send_email(to: string, body: string)` fonksiyonu çağrılabilir.

**API veya Veritabanı Sorguları Oluşturmak**  
Kullanıcılar, doğal dilde sordukları soruların biçimlendirilmiş bir sorguya veya API isteğine dönüştürülmesiyle bilgi bulabilirler. Buna örnek olarak, bir öğretmenin "Son ödevi tamamlayan öğrenciler kimler?" diye sorması verilebilir. Bu durumda `get_completed(student_name: string, assignment: int, current_status: string)` adında bir fonksiyon çağrılabilir.

**Yapılandırılmış Veri Oluşturmak**  
Kullanıcılar, bir metin bloğu veya CSV dosyasını kullanarak LLM’den önemli bilgileri çıkarmasını isteyebilir. Örneğin, bir öğrenci barış anlaşmalarıyla ilgili bir Wikipedia makalesini yapay zeka ile hazırlanmış bilgi kartlarına dönüştürebilir. Bu, `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` adlı bir fonksiyon kullanılarak yapılabilir.


## 2. İlk Fonksiyon Çağrınızı Oluşturma

Bir fonksiyon çağrısı oluşturma süreci 3 ana adımdan oluşur:
1. Fonksiyonlarınızın bir listesini ve bir kullanıcı mesajını kullanarak Chat Completions API'ını çağırmak
2. Modelin yanıtını okuyup bir işlem yapmak, yani bir fonksiyon veya API çağrısı gerçekleştirmek
3. Fonksiyonunuzdan gelen yanıtla birlikte Chat Completions API'ına tekrar çağrı yapmak ve bu bilgiyi kullanarak kullanıcıya bir yanıt oluşturmak


![Bir Fonksiyon Çağrısının Akışı](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.tr.png)


### Bir fonksiyon çağrısının öğeleri

#### Kullanıcı Girdisi

İlk adım, bir kullanıcı mesajı oluşturmaktır. Bu, bir metin girişinden alınan değeri dinamik olarak atayarak yapılabilir veya burada bir değer atayabilirsiniz. Eğer Chat Completions API ile ilk kez çalışıyorsanız, mesajın `role` (rol) ve `content` (içerik) kısımlarını tanımlamamız gerekir.

`role` değeri `system` (kuralları belirleyen), `assistant` (model) veya `user` (son kullanıcı) olabilir. Fonksiyon çağrısı için bunu `user` olarak ve örnek bir soru ile atayacağız.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Fonksiyon oluşturma.

Şimdi bir fonksiyon ve bu fonksiyonun parametrelerini tanımlayacağız. Burada sadece `search_courses` adlı bir fonksiyon kullanacağız ama birden fazla fonksiyon da oluşturabilirsiniz.

**Önemli** : Fonksiyonlar, sistem mesajına LLM'ye dahil edilir ve kullanılabilir token miktarınıza dahil edilir.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Tanımlar**

`name` - Çağrılmasını istediğimiz fonksiyonun adı.

`description` - Fonksiyonun nasıl çalıştığının açıklaması. Burada açık ve net olmak önemli.

`parameters` - Modelin yanıtında üretmesini istediğiniz değerlerin ve formatın listesi.

`type` - Özelliklerin saklanacağı veri tipi.

`properties` - Modelin yanıtı için kullanacağı belirli değerlerin listesi.

`name` - Modelin biçimlendirilmiş yanıtında kullanacağı özelliğin adı.

`type` - Bu özelliğin veri tipi.

`description` - Belirli özelliğin açıklaması.

**Opsiyonel**

`required` - Fonksiyon çağrısının tamamlanması için gerekli olan özellik.


### Fonksiyon çağrısı yapmak
Bir fonksiyon tanımladıktan sonra, şimdi onu Chat Completion API çağrısına dahil etmemiz gerekiyor. Bunu, isteğe `functions` ekleyerek yapıyoruz. Bu durumda `functions=functions` olarak ekleniyor.

Ayrıca `function_call` seçeneğini `auto` olarak ayarlama imkanı da var. Bu, fonksiyonun hangisinin çağrılacağına bizim karar vermemiz yerine, LLM’in kullanıcı mesajına göre uygun fonksiyonu seçmesine izin vereceğimiz anlamına gelir.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Şimdi yanıtı inceleyelim ve nasıl biçimlendirildiğine bakalım:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Burada fonksiyonun adının çağrıldığını ve kullanıcı mesajından, LLM'nin fonksiyonun argümanlarına uygun verileri bulabildiğini görebilirsiniz.


## 3. Bir Uygulamaya Fonksiyon Çağrılarını Entegre Etmek

LLM'den gelen biçimlendirilmiş yanıtı test ettikten sonra, artık bunu bir uygulamaya entegre edebiliriz.

### Akışı Yönetmek

Bunu uygulamamıza entegre etmek için şu adımları izleyelim:

Öncelikle, Open AI servislerine çağrı yapalım ve mesajı `response_message` adlı bir değişkende saklayalım.


In [ ]:
response_message = response.choices[0].message

Şimdi, Microsoft Learn API'sini çağırarak bir kurs listesi alacak olan fonksiyonu tanımlayacağız:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



En iyi uygulama olarak, modelin bir fonksiyon çağırmak isteyip istemediğine bakacağız. Ardından, mevcut fonksiyonlardan birini oluşturup çağrılan fonksiyonla eşleştireceğiz.
Sonrasında, fonksiyonun argümanlarını alıp bunları LLM'den gelen argümanlarla eşleştireceğiz.

Son olarak, fonksiyon çağrısı mesajını ve `search_courses` mesajından dönen değerleri ekleyeceğiz. Bu, LLM'nin kullanıcıya doğal bir dille yanıt vermesi için ihtiyaç duyduğu tüm bilgileri sağlar.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Kod Yarışması

Harika iş çıkardınız! Azure Open AI Fonksiyon Çağrısı hakkında öğrenmeye devam etmek için şunları yapabilirsiniz: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - Öğrencilerin daha fazla kurs bulmasına yardımcı olabilecek fonksiyonun daha fazla parametresini ekleyin. Kullanılabilir API parametrelerini burada bulabilirsiniz: 
 - Öğrenciden ana dili gibi daha fazla bilgi alan başka bir fonksiyon çağrısı oluşturun 
 - Fonksiyon çağrısı ve/veya API çağrısı uygun bir kurs döndürmediğinde hata yönetimi ekleyin



---

**Feragatname**:
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerde hata veya yanlışlıklar bulunabileceğini lütfen unutmayın. Belgenin orijinal diliyle hazırlanmış hali esas alınmalıdır. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından doğabilecek yanlış anlama veya yanlış yorumlamalardan sorumlu değiliz.
